[View in Colaboratory](https://colab.research.google.com/github/heumchri/erfnet_pytorch/blob/master/testPretrainedERFNet.ipynb)

In [1]:
import sys
sys.version
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
ls

datalab/  erfnet_pytorch/


In [0]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 

    100% |████████████████████████████████| 592.3MB 1.3MB/s 


In [0]:
!pip3 install numpy matplotlib torchvision Pillow visdom

    100% |████████████████████████████████| 61kB 2.3MB/s 
    100% |████████████████████████████████| 2.0MB 6.6MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [0]:
!git clone https://github.com/heumchri/erfnet_pytorch.git

Cloning into 'LightNet'...
remote: Counting objects: 342, done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 342 (delta 4), reused 0 (delta 0), pack-reused 329
Receiving objects: 100% (342/342), 89.82 MiB | 39.40 MiB/s, done.
Resolving deltas: 100% (144/144), done.


In [3]:
cd erfnet_pytorch/

/content/erfnet_pytorch


In [4]:
!mkdir datasets

mkdir: cannot create directory ‘datasets’: File exists


In [5]:
cd datasets/

/content/erfnet_pytorch/datasets


In [8]:
!wget --keep-session-cookies --save-cookies=cookies.txt --post-data 'username=heumchri&password=ctscpsdtst4heumchri&submit=Login' https://www.cityscapes-dataset.com/login/

--2018-06-06 09:23:04--  https://www.cityscapes-dataset.com/login/
Resolving www.cityscapes-dataset.com (www.cityscapes-dataset.com)... 139.19.217.8
Connecting to www.cityscapes-dataset.com (www.cityscapes-dataset.com)|139.19.217.8|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘index.html.3’

index.html.3            [ <=>                ]  24.88K  --.-KB/s    in 0.07s   

2018-06-06 09:23:04 (355 KB/s) - ‘index.html.3’ saved [25482]



In [7]:
!wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=1
!wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=3

--2018-06-06 09:20:27--  https://www.cityscapes-dataset.com/file-handling/?packageID=1
Resolving www.cityscapes-dataset.com (www.cityscapes-dataset.com)... 139.19.217.8
Connecting to www.cityscapes-dataset.com (www.cityscapes-dataset.com)|139.19.217.8|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.cityscapes-dataset.com/login/ [following]
--2018-06-06 09:20:27--  https://www.cityscapes-dataset.com/login/
Reusing existing connection to www.cityscapes-dataset.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘index.html.1’

index.html.1            [ <=>                ]  24.79K  --.-KB/s    in 0.05s   

2018-06-06 09:20:27 (469 KB/s) - ‘index.html.1’ saved [25383]

--2018-06-06 09:20:29--  https://www.cityscapes-dataset.com/file-handling/?packageID=3
Resolving www.cityscapes-dataset.com (www.cityscapes-dataset.com)... 139.19.217.8
Connecting to www.cityscapes-dataset.com (www.cityscapes-dataset.c

In [0]:
#!kill -9 -1